In [1]:
import Pkg; Pkg.activate(@__DIR__)
using Pkg; Pkg.instantiate()
using DifferentialEquations, LinearAlgebra, Statistics, Random, HypothesisTests
using StatsPlots, Plots, Distributions, KernelDensity, StatsBase, CSV, DataFrames
using LaTeXStrings, Measures, FileIO, JLD2

  Activating project at `~/Documents/Bachelor Thesis Stuff/bachelorsthesis`


In [2]:
# === Load Experimental Data ===
df = CSV.read("datasetsMA/nitrogenlim.csv", DataFrame)
dfs = hcat(df.Xa, df.Xi, df.N, df.S, df.FG, df.MA)
u0 = dfs[1, :]
t_exp = df.time
T = 40
tspan = (0.0, T)
dt = 0.01
tsteps = collect(0.0:dt:T)
Nt = length(tsteps)

# === Define ODE Model ===
function f!(du, u, p, t)
    # Unpack state and parameters
    μmax, KFG, KN, YXa_S, YXi_S, YXa_N, YP_S, ϕ, χacc, μ2max, qsplit_max, Ksuc, qpmax, KIP, KIN, KPFG, KFG2, σxa, σxi, σn, σs, σfg, σp = p
    Xact, Xinact, N, Suc, FruGlu, P = u

    # Ensure non-negative values
    ϵ = 1e-8  # Small positive value to avoid division by zero
    Xact_safe = max(Xact, ϵ)
    Xtot_safe = max(Xact + Xinact, ϵ)
    FruGlu_safe = max(FruGlu, ϵ)
    Suc_safe = max(Suc, ϵ)

    # Algebraic equations
    Xtot = Xact + Xinact
    N_int = 0.08 * N
    ratio = Xinact / Xact_safe
    expo_term = (ratio - ϕ) / χacc

    μ = μmax * FruGlu_safe / (FruGlu_safe + KFG + ϵ) * (N / (N+ KN + ϵ))
    μ2 = μ2max * FruGlu_safe / (FruGlu_safe + KFG2 + ϵ) * (1 - exp(expo_term)) * KIN / (KIN + N + ϵ)
    qsplit = qsplit_max * Suc_safe / (Suc_safe + Ksuc + ϵ)
    qp = qpmax * FruGlu_safe / (FruGlu_safe + KPFG + ϵ) *
         (KIP / (KIP + N_int / Xtot_safe + ϵ)) * KIN / (KIN + N + ϵ)

    du[1] = μ * Xact
    du[2] = μ2 * Xact
    du[3] = - (μ / YXa_N) * Xact
    du[4] = - qsplit * Xact
    du[5] = (qsplit - μ / YXa_S - μ2 / YXi_S - qp / YP_S) * Xact
    du[6] = qp * Xact
end

# === Define Noise Function ===
function noise!(du, u, p, t)
    Xact, Xinact, N, Suc, FruGlu, P = u
    μmax, KFG, KN, YXa_S, YXi_S, YXa_N, YP_S, ϕ, χacc, μ2max, qsplit_max, Ksuc, qpmax, KIP, KIN, KPFG, KFG2, σxa, σxi, σn, σs, σfg, σp = p
    du[1] = σxa * FruGlu / (FruGlu + (KFG)) * Xact
    du[2] = σxi * FruGlu / (FruGlu + (KFG2)) * Xact
    du[3] = σn * N / (N + KN) * Xact
    du[4] = σs * Suc / (Suc + Ksuc) * Xact
    du[5] = σfg * FruGlu / (FruGlu + (Ksuc + KFG + KFG2 + KPFG)) * Xact
    du[6] = σp * FruGlu / (FruGlu + (KPFG)) * Xact
end

# === Parameters ===
params = [
    0.125,  # 1. μmax
    0.147,  # 2. KFG
    3.8e-5,  # 3. KN
    0.531,  # 4. YXa_S
    0.799,  # 5. YXi_S
    9.428,  # 6. YXa_N
    0.508,  # 7. YP_S
    1.56,  # 7. ϕ
    0.3,  # 8. χacc
    0.125,  # 9. μ2max
    1.985,  # 10. qsplit_max
    0.00321,  # 11. Ksuc
    0.095,  # 12. qpmax
    1.47e-1,  # 13. KIP
    1.47e-2,  # 14. KIN
    0.0175,  # 15. KPFG
    3.277,  # 16. KFG2
    5e-2,   # 17. σxa
    5e-2,   # 18. σxi
    1e-2,   # 19. σn
    5e-2,   # 20. σs
    5e-2,   # 21. σfg
    5e-2    # 22. σp
]

# === Solve the ODE ===
odeprob = ODEProblem(f!, u0, tspan, params)
odesol = solve(odeprob, Rosenbrock23(), saveat=tsteps, abstol=1e-8, reltol=1e-6)

# === Extract ODE solution ===
Xa_ode = odesol[1, :]
Xi_ode = odesol[2, :]
N_ode  = odesol[3, :]
Suc_ode = odesol[4, :]
FG_ode = odesol[5, :]
P_ode = odesol[6, :]

# === Important Information ===
println(Nt, " time steps")
println("Initial conditions: ", u0)

4001 time steps
Initial conditions: [1.85, 0.0, 0.762155059, 61.957, 12.30333333, 0.0]


### Simulation Set 1

In [3]:
# Load the saved ensemble solution
ensemble_sol = load("ensembledata/ensemble_sol.jld2", "ensemble_sol")

# Extract results
Xact_mat = hcat([sol[1, :] for sol in ensemble_sol]...)
Xinact_mat = hcat([sol[2, :] for sol in ensemble_sol]...)
N_mat = hcat([sol[3, :] for sol in ensemble_sol]...)
Suc_mat = hcat([sol[4, :] for sol in ensemble_sol]...)
FruGlu_mat = hcat([sol[5, :] for sol in ensemble_sol]...)
P_mat = hcat([sol[6, :] for sol in ensemble_sol]...)

# Ensemble Summary
summ = EnsembleSummary(ensemble_sol, tsteps)

EnsembleSolution Solution of length 4001 with uType:
Float64

### Simulation Set 2

In [4]:
# Load the saved ensemble solution
ensemble_sol2 = load("ensembledata/ensemble_sol2.jld2", "ensemble_sol")

# Extract results
Xact_mat2 = hcat([sol[1, :] for sol in ensemble_sol2]...)
Xinact_mat2 = hcat([sol[2, :] for sol in ensemble_sol2]...)
N_mat2 = hcat([sol[3, :] for sol in ensemble_sol2]...)
Suc_mat2 = hcat([sol[4, :] for sol in ensemble_sol2]...)
FruGlu_mat2 = hcat([sol[5, :] for sol in ensemble_sol2]...)
P_mat2 = hcat([sol[6, :] for sol in ensemble_sol2]...)

# Ensemble Summary
summ2 = EnsembleSummary(ensemble_sol, tsteps)

EnsembleSolution Solution of length 4001 with uType:
Float64

### LSMC

In [5]:
# === Laguerre basis functions up to degree 3 ===
d = 3  # Degree of the Laguerre polynomial
M = 5000  # Number of simulations

function laguerre_design_matrix(y::Vector{Float64}, d::Int)
    Φ = zeros(length(y), d + 1)
    for i in 1:length(y)
        Φ[i,1] = 1.0
        if d >= 1
            Φ[i,2] = 1 - y[i]
        end
        if d >= 2
            Φ[i,3] = 1 - 2*y[i] + 0.5*y[i]^2
        end
        if d >= 3
            Φ[i,4] = 1 - 3*y[i] + 1.5*y[i]^2 - (1/6)*y[i]^3
        end
    end
    return Φ
end

# === Reward function ===
thresh = 0.05 *  (u0[4] + u0[5]) # Threshold for stopping condition: 5% of initial substrates (Suc + FruGlu)
reward(s) = -(s-thresh)^2

# === Important Information ===
println("Threshold for stopping condition: ", thresh)

Threshold for stopping condition: 3.7130166664999997


In [6]:
# === Calculate the reward at all time steps ===
rewards = reward.(FruGlu_mat)

# === Filter the paths that have not reached the threshold at the last time step (in-the-money) ===
valid_paths = findall(i -> FruGlu_mat[Nt, i] < thresh, 1:M)
if isempty(valid_paths)
    error("No valid paths found at the last time step.")
end

# === Prepare the matrices for backward induction ===
# Only keep the valid paths for the backward induction
s_valid = FruGlu_mat[:, valid_paths] 
rewards_valid = rewards[:, valid_paths]
τ = fill(Nt, length(valid_paths)) 

# === The backward induction ===
for n in (Nt-1):-1:2
    itm_idx = findall(i -> s_valid[n, i] < thresh, 1:length(valid_paths)) 
    s_itm = s_valid[:, itm_idx]  # Stores all in-the-money paths
    s_now = s_itm[n, :]  # Current state for itm paths
    s_future = s_valid[n+1, itm_idx] # Future state for itm paths
    reward_now = reward.(s_now)  # Rewards at the current time step
    reward_future = collect(Float64, reward.(s_future))
    
    # Regression and saving the coefficients
    ϕ = laguerre_design_matrix(s_now, d)
    β = ϕ \ reward_future  # Solve the linear system to find coefficients
    E = ϕ * β  # Calculate the expected reward for the valid paths

    for (k,i) in enumerate(itm_idx)
        if E[k] ≤ reward_now[k]  # If the expected reward is less than or equal to the current reward
            τ[k] = min(n, τ[k])  # Update the stopping time for this path
            rewards_valid[n, k] = reward_now[k]  # Keep the current reward
        else
            rewards_valid[n, k] = rewards_valid[n+1, k]  # Otherwise, carry forward
        end
    end
end

@show size(s_valid)

size(s_valid) = (4001, 3006)


(4001, 3006)

In [7]:
# === Calculate the reward at all time steps ===
rewards2 = reward.(FruGlu_mat2)

# === Filter the paths that have not reached the threshold at the last time step (in-the-money) ===
valid_paths2 = findall(i -> FruGlu_mat2[Nt, i] < thresh, 1:M)
if isempty(valid_paths2)
    error("No valid paths found at the last time step.")
end

# === Prepare the matrices for backward induction ===
# Only keep the valid paths for the backward induction
s_valid2 = FruGlu_mat2[:, valid_paths2] 
rewards_valid2 = rewards2[:, valid_paths2]
τ2 = fill(Nt, length(valid_paths2)) 

# === The backward induction ===
for n in (Nt-1):-1:2
    itm_idx = findall(i -> s_valid2[n, i] < thresh, 1:length(valid_paths2)) 
    s_itm = s_valid2[:, itm_idx]  # Stores all in-the-money paths
    s_now = s_itm[n, :]  # Current state for itm paths
    s_future = s_valid2[n+1, itm_idx] # Future state for itm paths
    reward_now = reward.(s_now)  # Rewards at the current time step
    reward_future = collect(Float64, reward.(s_future))
    
    # Regression and saving the coefficients
    ϕ = laguerre_design_matrix(s_now, d)
    β = ϕ \ reward_future  # Solve the linear system to find coefficients
    E = ϕ * β  # Calculate the expected reward for the valid paths

    for (k,i) in enumerate(itm_idx)
        if E[k] ≤ reward_now[k]  # If the expected reward is less than or equal to the current reward
            τ2[k] = min(n, τ2[k])  # Update the stopping time for this path
            rewards_valid2[n, k] = reward_now[k]  # Keep the current reward
        else
            rewards_valid2[n, k] = rewards_valid2[n+1, k]  # Otherwise, carry forward
        end
    end
end

@show size(s_valid2)

size(s_valid2) = (4001, 2934)


(4001, 2934)

### Statistics

In [20]:
# === Identify when the deterministic case would have reached the threshold ===
ode_hit_index = findfirst(x -> x <= thresh, FG_ode)
ode_hit_time = tsteps[ode_hit_index]

# === Compile the optimal stopping times ===
stop_idx   = copy(τ)          
stop_data  = tsteps[stop_idx]
stop_times_avg = mean(stop_data)

# === Compile the optimal stopping times ===
stop_idx2   = copy(τ2)          
stop_data2  = tsteps[stop_idx2]
stop_times_avg2 = mean(stop_data2)

# === Prepare a common x‐range for the curves ===
xmin = minimum([minimum(stop_data), minimum(stop_data2)])
xmax = 42 # maximum([maximum(hit_data), maximum(stop_data)])
xs   = range(xmin, xmax; length=200)

# === Kernel density estimation for the hit and stop times ===
kde_stop = kde(stop_data)
kde_stop2 = kde(stop_data2)

# === Plot the results ===
plot(xlabel="Time / h", ylabel="Density", legend=:topleft, linewidth=2, size=(900, 600), fontfamily="Computer Modern", margins=5mm,
     xlims=(xmin, xmax), ylims=(0, 0.09), legendfont = 12, tickfont = 11, guidefont = 16)
plot!(xs, pdf(kde_stop, xs), label="KDE of Stop Times (Set 1)", color=:blue, linewidth=2)
vline!([stop_times_avg], label="Average Stop Time (Set 1)", color=:blue, linestyle=:dot, linewidth=2)
plot!(xs, pdf(kde_stop2, xs), label="KDE of Stop Times (Set 2)", color=:red, linewidth=2)
vline!([stop_times_avg2], label="Average Stop Time (Set 2)", color=:red, linestyle=:dot, linewidth=2)
vline!([ode_hit_time], label="ODE Hit Time", color=:black, linestyle=:dashdot, linewidth=2)

savefig("Figures/2KDE_plots.pdf")


"/Users/rafif/Documents/Bachelor Thesis Stuff/bachelorsthesis/Figures/2KDE_plots.pdf"